# Bash in Your Notebook
At work recently, I had to call an internal REST API.  To try the API out, I fired up my Ubuntu in my Windows Subsystem for Linux and ran a cURL command to try out the interface.  That went well, so I created a new Jupyter Notebook in which to call the API--I wanted to take the load them into a pandas dataframe, and create a chart across the data.

So, I called the API with the [requests](https://realpython.com/python-requests/) and promptly received a SSL "bad handshake" error.  [Like many others](https://stackoverflow.com/questions/28667684/python-requests-getting-sslerror), I struggled to resolve this error.  Clearly, the server hosting that API was malconfigured in some way.  However, I didn't own the server and had no real recourse to get the issue fixed, so I decided to call cURL directly from my notebook and this led me to the [bash magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-bash).

With the *bash* magic command, you can mark tell Jupyter Notebook to run all the commands in your cell as if you were executing them at a bash command prompt...even if you're running Jupyter Notebook on a Windows operating system.  How cool is that?!  Furthermore, with the *out* argument, you can pipe all your cell output to a variable for easy processing.  Check this out:<br><br>
(Note: I'm using the free lyrics API from [lyrics.ovh](http://docs.lyricsovh.apiary.io/) in my examples below)

Get some lyrics and write them to the variable *lyrics1*:

In [7]:
%%bash --out lyrics1
curl https://private-anon-cd823708f8-lyricsovh.apiary-proxy.com/v1/the%20beatles/here%20comes%20the%20sun

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   843  100   843    0     0    126      0  0:00:06  0:00:06 --:--:--   213


In [8]:
lyrics1

'{"lyrics":"Here comes the sun\\nHere comes the sun\\nAnd I say\\nIt\'s all right\\n\\nLittle darlin\'\\nIt\'s been a long cold lonely winter\\nLittle darlin\'\\nIt feels like years since it\'s been here\\nHere comes the sun\\nHere comes the sun, and I say\\nIt\'s all right\\n\\nLittle darlin\'\\nThe smiles returning to their faces\\nLittle darlin\'\\nIt seems like years since it\'s been here\\nHere comes the sun\\nHere comes the sun, and I say\\nIt\'s all right\\n\\nSun, sun, sun, here it comes\\nSun, sun, sun, here it comes\\nSun, sun, sun, here it comes\\nSun, sun, sun, here it comes\\nSun, sun, sun, here it comes\\n\\nLittle darlin\'\\nI feel that ice is slowly melting\\nLittle darlin\'\\nIt seems like years since it\'s been clear\\nHere comes the sun\\nHere comes the sun, and I say\\nIt\'s all right\\n\\nHere comes the sun\\nHere comes the sun\\nIt\'s all right\\nIt\'s all right"}'

Interestingly, *lyrics1* is typed as a string even though it's really a JSON object:

In [9]:
type(lyrics1)

str

Fortunately, the *json.loads* function can make easy work of this string:

In [14]:
import json
json_lyrics1 = json.loads(lyrics1)
print(json_lyrics1['lyrics'])

Here comes the sun
Here comes the sun
And I say
It's all right

Little darlin'
It's been a long cold lonely winter
Little darlin'
It feels like years since it's been here
Here comes the sun
Here comes the sun, and I say
It's all right

Little darlin'
The smiles returning to their faces
Little darlin'
It seems like years since it's been here
Here comes the sun
Here comes the sun, and I say
It's all right

Sun, sun, sun, here it comes
Sun, sun, sun, here it comes
Sun, sun, sun, here it comes
Sun, sun, sun, here it comes
Sun, sun, sun, here it comes

Little darlin'
I feel that ice is slowly melting
Little darlin'
It seems like years since it's been clear
Here comes the sun
Here comes the sun, and I say
It's all right

Here comes the sun
Here comes the sun
It's all right
It's all right


So, what if you need to call your bash commands multiple times, say in a loop?  Well, here's one hack I came up with: do in-line calls to the [shell](https://ipython.readthedocs.io/en/stable/interactive/python-ipython-diff.html#shell-assignment).

In [17]:
song_list = ['yesterday', 'yellow%20submarine', 'eleanor%20rigby']
song_lyrics = []

for song in song_list:
    lyric = !wsl curl -s 'https://private-anon-cd823708f8-lyricsovh.apiary-proxy.com/v1/the%20beatles/{song}'
    song_lyrics.append(lyric)
    
song_lyrics

[['{"lyrics":"Yesterday\\nAll my troubles seemed so far away\\nNow it looks as though they\'re here to stay\\nOh, I believe in yesterday\\n\\nSuddenly\\nI\'m not half the man I used to be\\nThere\'s a shadow hanging over me\\nOh, yesterday came suddenly\\n\\nWhy she had to go, I don\'t know\\nShe wouldn\'t say\\nI said something wrong\\nNow I long for yesterday\\n\\nYesterday\\nLove was such an easy game to play\\nNow I need a place to hide away\\nOh, I believe in yesterday\\n\\nWhy she had to go, I don\'t know\\nShe wouldn\'t say\\nI said something wrong\\nNow I long for yesterday\\n\\nYesterday\\nLove was such an easy game to play\\nNow I need a place to hide away\\nOh, I believe in yesterday"}'],
 ['{"lyrics":"In the town where I was born\\nLived a man who sailed to sea\\nAnd he told us of his life\\nIn the land of submarines\\nSo we sailed on to the sun\\nTill we found a sea of green\\nAnd we lived beneath the waves\\nIn our yellow submarine\\n\\nWe all live in a yellow submarine\\

Here are a few things to note with my shell operation:
 - Since my operating system is Windows 10, I'm actually shelling out to the Windows command shell, not bash.  However, since I have [WSL](https://docs.microsoft.com/en-us/windows/wsl/faq) installed on my machine, I can use [wsl.exe](https://docs.microsoft.com/en-us/windows/wsl/interop) to run commands in that shell.  So, I calling a shell within a shell to ultimately execute my bash command.
 - With the braces syntax, I can pass the value of my *song* variable to my shell command.
 - I pass the *silent* argument (-s) to cURL to suppress the noise cURL would normally send back to Jupyter Notebook.  This allows me to pass just the JSON response to my variable *lyric*.

Let's take a closer look of what my shell commands return:

In [18]:
type(song_lyrics[0])

IPython.utils.text.SList

Interesting.  Unlike the *string* type that the *bash* magic command returned, the shell command returned a [SList](https://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.text.html#IPython.utils.text.SList).  Basically, a list of strings.  I should be able to join those lists together, though, and then convert them to JSON with the loads function:

In [19]:
song_list = ['yesterday', 'yellow%20submarine', 'eleanor%20rigby']
song_lyrics = []

for song in song_list:
    lyric = !wsl curl -s 'https://private-anon-cd823708f8-lyricsovh.apiary-proxy.com/v1/the%20beatles/{song}'
    json_lyrics = json.loads(''.join(lyric))
    song_lyrics.append(json_lyrics)
    
song_lyrics

[{'lyrics': "Yesterday\nAll my troubles seemed so far away\nNow it looks as though they're here to stay\nOh, I believe in yesterday\n\nSuddenly\nI'm not half the man I used to be\nThere's a shadow hanging over me\nOh, yesterday came suddenly\n\nWhy she had to go, I don't know\nShe wouldn't say\nI said something wrong\nNow I long for yesterday\n\nYesterday\nLove was such an easy game to play\nNow I need a place to hide away\nOh, I believe in yesterday\n\nWhy she had to go, I don't know\nShe wouldn't say\nI said something wrong\nNow I long for yesterday\n\nYesterday\nLove was such an easy game to play\nNow I need a place to hide away\nOh, I believe in yesterday"},
 {'lyrics': 'In the town where I was born\nLived a man who sailed to sea\nAnd he told us of his life\nIn the land of submarines\nSo we sailed on to the sun\nTill we found a sea of green\nAnd we lived beneath the waves\nIn our yellow submarine\n\nWe all live in a yellow submarine\nYellow submarine, yellow submarine\nWe all live 

Sweet!  So now I have a list of JSON objects for the song lyrics I want:

In [20]:
print(song_lyrics[0]['lyrics'])

Yesterday
All my troubles seemed so far away
Now it looks as though they're here to stay
Oh, I believe in yesterday

Suddenly
I'm not half the man I used to be
There's a shadow hanging over me
Oh, yesterday came suddenly

Why she had to go, I don't know
She wouldn't say
I said something wrong
Now I long for yesterday

Yesterday
Love was such an easy game to play
Now I need a place to hide away
Oh, I believe in yesterday

Why she had to go, I don't know
She wouldn't say
I said something wrong
Now I long for yesterday

Yesterday
Love was such an easy game to play
Now I need a place to hide away
Oh, I believe in yesterday


For more on the bash magic command, check out [this excellent article](http://mmcdan.github.io/posts/interacting-with-the-shell-via-jupyter-notebook/).